In [21]:
import pandas as pd
import cudf
from sklearn.neighbors import KNeighborsClassifier
#from cuml.dask.neighbors import KNeighborsClassifier
#from dask_ml.model_selection import GridSearchCV
#from dask_ml.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import dask.dataframe as dd
from sklearn.model_selection import GridSearchCV
from joblib import dump, load

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

from dask_cuda import LocalCUDACluster
from dask.distributed import Client, wait

In [22]:
#cluster = LocalCUDACluster()
#client = Client(cluster)

In [23]:
df = pd.read_csv('../global_preprocessed_without_onehot.csv', encoding='ISO-8859-1')
#df = cudf.read_csv('../global_preprocessed.csv')


row_count = df.shape[0]
column_count = df.shape[1]
print(f'The DataFrame has {row_count} rows.')
print(f'The DataFrame has {column_count} columns.')
df

The DataFrame has 171280 rows.
The DataFrame has 10 columns.


,iyear,extended,region,nkill,nwound,attacktype1,nperps,suicide,success,gname
0,-2.952239,0,2,0.095379,0.000000,1,-0.048337,0,1,0
1,-2.952239,0,1,0.000000,0.000000,6,0.290024,0,1,1
2,-2.952239,0,5,0.095379,0.000000,1,-0.048337,0,1,2
3,-2.952239,0,8,0.000000,0.000000,3,-0.048337,0,1,2
4,-2.952239,0,4,0.000000,0.000000,7,-0.048337,0,1,2
...,...,...,...,...,...,...,...,...,...,...
171275,0.605588,0,11,0.095379,0.056697,2,0.000000,0,1,2656
171276,0.605588,0,10,0.190758,0.198440,3,0.000000,0,1,592
171277,0.605588,0,5,0.000000,0.000000,7,0.000000,0,1,2926
171278,0.605588,0,6,0.000000,0.000000,3,0.000000,0,0,2


In [24]:
x = df.drop("extended", axis=1)
y = df['extended']
#x = dd.from_pandas(x, npartitions=2)
#y = dd.from_pandas(y, npartitions=2)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, shuffle=True)

In [25]:
#knn = KNeighborsClassifier()
knn = GridSearchCV(estimator= KNeighborsClassifier(), 
                   param_grid={'n_neighbors': [10, 15], 'algorithm': ['ball_tree', 'brute']},
                   n_jobs=-1, verbose=3, scoring='precision_macro', cv=5)

over = SMOTE(sampling_strategy="auto")   # oversample minority class 
under = RandomUnderSampler(sampling_strategy=0.1) # undersample majority class 
steps = [('u', under), ('o', over)]
pipeline = Pipeline(steps=steps)

# transform the dataset
x_train, y_train = pipeline.fit_resample(x_train, y_train)


In [26]:
df1 = y_train[y_train>0]
row_count = y_train.shape[0]
print(f'The DataFrame (extended == 0) has {row_count} rows.')
df0 = y_train[y_train==0]
row_count = df0.shape[0]
print(f'The DataFrame (extended > 0) has {row_count} rows.')

The DataFrame (extended == 0) has 119020 rows.
The DataFrame (extended > 0) has 59510 rows.


In [27]:
knn.fit(x_train, y_train)
knn.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


{'algorithm': 'ball_tree', 'n_neighbors': 10}

In [28]:
dump(knn, "knn_model.joblib")

['knn_model.joblib']

In [29]:
#client.close()